<h1>Part 1. Sequence Tagging: NER </h1>  
Build an NER classifier using pretrained word embeddings. Specifically, NER aims to identify named
entities (words or phrases) from each sentence given. To do that, we can formulate this task to a
sequence tagging problem where each word in the sentence will be assigned a label. You can use the
IO, BIO or any other sequence tagging scheme of your choice.


<h2> 1.1 Word Embedding </h2>

In [3]:
!pip install --upgrade gensim

In [2]:
import gensim.downloader

# Downlaod 'word2vec-google-news-300' embeddings
google_news_vectors = gensim.downloader.load('word2vec-google-news-300')

<h2> Question 1.1</h2>

In [3]:
student = google_news_vectors.most_similar('student')
Apple = google_news_vectors.most_similar('Apple')
apple = google_news_vectors.most_similar('apple')

In [4]:
# Present most similar words for each word
for word in student:
    print(word)

('students', 0.7294867038726807)
('Student', 0.6706662774085999)
('teacher', 0.6301366090774536)
('stu_dent', 0.6240993142127991)
('faculty', 0.6087332963943481)
('school', 0.6055627465248108)
('undergraduate', 0.6020305752754211)
('university', 0.600540041923523)
('undergraduates', 0.5755698680877686)
('semester', 0.573759913444519)


In [5]:
for word in Apple:
    print(word)

('Apple_AAPL', 0.7456986308097839)
('Apple_Nasdaq_AAPL', 0.7300410270690918)
('Apple_NASDAQ_AAPL', 0.717508852481842)
('Apple_Computer', 0.7145972847938538)
('iPhone', 0.6924266219139099)
('Apple_NSDQ_AAPL', 0.6868603229522705)
('Steve_Jobs', 0.6758421659469604)
('iPad', 0.6580768823623657)
('Apple_nasdaq_AAPL', 0.6444970369338989)
('AAPL_PriceWatch_Alert', 0.6439753174781799)


In [6]:
for word in apple:
    print(word)

('apples', 0.720359742641449)
('pear', 0.6450697183609009)
('fruit', 0.6410146355628967)
('berry', 0.6302295327186584)
('pears', 0.613396167755127)
('strawberry', 0.6058260798454285)
('peach', 0.6025872826576233)
('potato', 0.5960935354232788)
('grape', 0.5935863852500916)
('blueberry', 0.5866668224334717)


In [7]:
# Create a dictionary of most similar words
most_similar_word_dict = {}
most_similar_word_dict['student'] = student[0]
most_similar_word_dict['Apple'] = Apple[0]
most_similar_word_dict['apple'] = apple[0]

# Print the dictionary
question_list = ['(a)', '(b)', '(c)']
index = 0
for key,item in most_similar_word_dict.items():
    print(f"{question_list[index]} The most similar word to '{key}' is {item[0]} with a cosine similarity score of {item[1]}")
    index += 1

(a) The most similar word to 'student' is students with a cosine similarity score of 0.7294867038726807
(b) The most similar word to 'Apple' is Apple_AAPL with a cosine similarity score of 0.7456986308097839
(c) The most similar word to 'apple' is apples with a cosine similarity score of 0.720359742641449


<h2> 1.2 Data </h2>

   
Train - eng.train  
Development -eng.testa     
Test - eng.testb

In [1]:
# Read the files
with open('eng.train', 'r') as f:
    train = f.readlines()

with open('eng.testa', 'r') as f:
    development = f.readlines()

with open('eng.testb', 'r') as f:
    test = f.readlines()


<h2> Question 1.2 </h2>

(a) Describe the size (number of sentences) of the training, development and test file for CoNLL2003.
Specify the complete set of all possible word labels based on the tagging scheme (IO, BIO,
etc.) you chose.


In [9]:
# Print the number of sentences in each file
# Note: There is a sentence when the line is empty

# files = [train, development, test]
# for file in files:
#     num_sentences= 0
#     for line in file:
#         if line == '\n':
#             num_sentences += 1
    
#     print(f"The number of sentences in {file} is {num_sentences}")

num_sentences = 0
for line in train:
    if line == '\n':
        num_sentences += 1

print(f"The number of sentences in training file is {num_sentences}")

num_sentences = 0
for line in development:
    if line == '\n':
        num_sentences += 1

print(f"The number of sentences in development file is {num_sentences}")

num_sentences = 0
for line in test:
    if line == '\n':
        num_sentences += 1

print(f"The number of sentences in test file is {num_sentences}")

The number of sentences in training file is 14986
The number of sentences in development file is 3465
The number of sentences in test file is 3683


In [10]:
set_of_all_word_labels = set()
files = [train, development, test]
for file in files:
    for line in file:
        if line != '\n':
            line = line.strip()
            word_label = line.split()[-1]
            set_of_all_word_labels.add(word_label)

# Print the set of all word labels
print(set_of_all_word_labels)

{'I-MISC', 'B-ORG', 'I-LOC', 'I-ORG', 'I-PER', 'B-MISC', 'O', 'B-LOC'}


(b) Choose an example sentence from the training set of CoNLL2003 that has at least two named
entities with more than one word. Explain how to form complete named entities from the label
for each word, and list all the named entities in this sentence.

Code to store the word and its respective label for each sentence in a dictionary

In [48]:
# Function to store the word and its label for each sentence in a dictionary
from collections import defaultdict
def get_word_label_dict(file):
    word_label_dict = defaultdict(dict)
    index = 1
    for line in file:
        if line != '\n':
            line = line.strip()
            word_label = line.split()[-1]
            word = line.split()[0]
            word_label_dict[index][word] = word_label
        if line == '\n': 
            index += 1
    return word_label_dict
# word_label_dict = defaultdict(dict)
# index = 1
# for line in train:
#     if line != '\n':
#         line = line.strip()
#         word_label = line.split()[-1]
#         word = line.split()[0]
#         # word_label_dict[word][word_label] = word_label_dict[word].get(word_label, 0) + 1
#         word_label_dict[index][word] = word_label
#     if line == '\n': 
#         index += 1

Code to process the word_label_dict generated above to convert the IO scheme to BIO scheme

In [54]:
def convert_IO_to_BIO(file, file_type):
    word_label_dict = get_word_label_dict(file)
    stop = 0
    IO_tagging = ['I-ORG', 'I-PER', 'I-LOC', 'I-MISC']
    IO_to_BIO_dict = {'I-ORG': 'B-ORG', 'I-PER': 'B-PER', 'I-LOC': 'B-LOC', 'I-MISC': 'B-MISC'}
    for sentence_no, word_and_word_label in word_label_dict.items():
        current_IO_tag = ''
        print(f"Sentence number: {sentence_no} word_and_word_label: {word_and_word_label}")
        for word, word_label in word_and_word_label.items():
            if word_label in IO_tagging:
                if word_label == current_IO_tag:
                    pass
                if current_IO_tag == '':
                    current_IO_tag = word_label
                    word_label_dict[sentence_no][word] = IO_to_BIO_dict[word_label]
                    print(f"Changed {word_label} to {word_label_dict[sentence_no][word]} for {word}")
                    
            else:
                current_IO_tag = ''
        print(f"Sentence number: {sentence_no} word_and_word_label: {word_and_word_label}")
        stop += 1
        if stop == 5:
            break
    # save the word_label_dict to a text file
    file = f"{file_type}_BIO.txt"
    with open(file, 'w') as f:
        for sentence_no, word_and_word_label in word_label_dict.items():
            for word, word_label in word_and_word_label.items():
                f.write(f"{word} {word_label}\n")
            f.write('\n')

convert_IO_to_BIO(train, 'train')


Sentence number: 1 word_and_word_label: {'EU': 'I-ORG', 'rejects': 'O', 'German': 'I-MISC', 'call': 'O', 'to': 'O', 'boycott': 'O', 'British': 'I-MISC', 'lamb': 'O', '.': 'O'}
Changed I-ORG to B-ORG for EU
Changed I-MISC to B-MISC for German
Changed I-MISC to B-MISC for British
Sentence number: 1 word_and_word_label: {'EU': 'B-ORG', 'rejects': 'O', 'German': 'B-MISC', 'call': 'O', 'to': 'O', 'boycott': 'O', 'British': 'B-MISC', 'lamb': 'O', '.': 'O'}
Sentence number: 2 word_and_word_label: {'Peter': 'I-PER', 'Blackburn': 'I-PER'}
Changed I-PER to B-PER for Peter
Sentence number: 2 word_and_word_label: {'Peter': 'B-PER', 'Blackburn': 'I-PER'}
Sentence number: 3 word_and_word_label: {'BRUSSELS': 'I-LOC', '1996-08-22': 'O'}
Changed I-LOC to B-LOC for BRUSSELS
Sentence number: 3 word_and_word_label: {'BRUSSELS': 'B-LOC', '1996-08-22': 'O'}
Sentence number: 4 word_and_word_label: {'The': 'O', 'European': 'I-ORG', 'Commission': 'I-ORG', 'said': 'O', 'on': 'O', 'Thursday': 'O', 'it': 'O', 'di

Code to process the word_label_dict generated above to extract the multi-word named entities for each sentence

In [25]:
# Process word_label_dict to get the multi-word named entities for each sentence
named_entity_list = ['I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']
consective_named_entity_words = 0
previous_named_entity_word_label = ''
consective_named_entity_words_dict = defaultdict(list)
current_sentence_consective_named_entity_words_dict = defaultdict()
for sentence_no, word_and_word_label in word_label_dict.items():
    for word, word_label in word_and_word_label.items():
        if word_label in named_entity_list:
            if word_label == previous_named_entity_word_label:
                consective_named_entity_words += 1
                current_sentence_consective_named_entity_words_dict[word] = word_label

            if previous_named_entity_word_label == '':
                consective_named_entity_words += 1
                previous_named_entity_word_label = word_label
                current_sentence_consective_named_entity_words_dict[word] = word_label
        
        else:
            previous_named_entity_word_label = ''
            if consective_named_entity_words >= 2:
                consective_named_entity_words_dict[sentence_no].append([current_sentence_consective_named_entity_words_dict.items()])

            current_sentence_consective_named_entity_words_dict = defaultdict()
            consective_named_entity_words = 0
            previous_named_entity_word_label = ''

print(consective_named_entity_words_dict)


defaultdict(<class 'list'>, {3: [[dict_items([('Peter', 'I-PER'), ('Blackburn', 'I-PER')])]], 4: [[dict_items([('European', 'I-ORG'), ('Commission', 'I-ORG')])]], 5: [[dict_items([('European', 'I-ORG'), ('Union', 'I-ORG')])], [dict_items([('Werner', 'I-PER'), ('Zwingmann', 'I-PER')])]], 6: [[dict_items([('Nikolaus', 'I-PER'), ('van', 'I-PER'), ('der', 'I-PER'), ('Pas', 'I-PER')])]], 7: [[dict_items([('European', 'I-ORG'), ('Union', 'I-ORG')])]], 8: [[dict_items([('Franz', 'I-PER'), ('Fischler', 'I-PER')])]], 9: [[dict_items([('Bovine', 'I-MISC'), ('Spongiform', 'I-MISC'), ('Encephalopathy', 'I-MISC')])]], 11: [[dict_items([('Loyola', 'I-PER'), ('de', 'I-PER'), ('Palacio', 'I-PER')])]], 17: [[dict_items([('Welsh', 'I-ORG'), ('National', 'I-ORG'), ('Farmers', 'I-ORG'), ("'", 'I-ORG'), ('Union', 'I-ORG')])], [dict_items([('John', 'I-PER'), ('Lloyd', 'I-PER'), ('Jones', 'I-PER')])], [dict_items([('BBC', 'I-ORG'), ('radio', 'I-ORG')])]], 24: [[dict_items([('Jimi', 'I-PER'), ('Hendrix', 'I-P

Code to print the multi-word named entities for each sentence in the same line of the print output

In [103]:
# Print the multi-word named entities for each sentence
combined_word_label = ''
combined_word_label_list = []
num_named_entity_words_more_than_one_word = 0
for key, value in consective_named_entity_words_dict.items():
    # Iterate through dict_item object which consists of all of the word and its label
    for i in range(len(value)):
        # Convert to list to iterate through the word and its label
        word_and_word_label_list = list(value[i][0])
        for word_and_word_label in word_and_word_label_list:
            # Joining the word and its label for a multi-word named entity
            combined_word_label += word_and_word_label[0] + ' ' + word_and_word_label[1] + ' '
        
        # Append the multi-word named entities to a list
        combined_word_label_list.append(combined_word_label)

        num_named_entity_words_more_than_one_word += 1

        # Print the multi-word named entities if there are at least 2 multi-words in each sentence
        if num_named_entity_words_more_than_one_word >= 2:
            print(f"Sentence number: {key}")
            for i in range(len(combined_word_label_list)):
                print(combined_word_label_list[i])
            print('\n')
        combined_word_label = ''

    num_named_entity_words_more_than_one_word = 0
    combined_word_label_list = []
        

Sentence number: 5
European I-ORG Union I-ORG 
Werner I-PER Zwingmann I-PER 


Sentence number: 17
Welsh I-ORG National I-ORG Farmers I-ORG ' I-ORG Union I-ORG 
John I-PER Lloyd I-PER Jones I-PER 


Sentence number: 17
Welsh I-ORG National I-ORG Farmers I-ORG ' I-ORG Union I-ORG 
John I-PER Lloyd I-PER Jones I-PER 
BBC I-ORG radio I-ORG 


Sentence number: 33
Taiwan I-LOC Strait I-LOC 
Lien I-PER Chan I-PER 


Sentence number: 34
Foreign I-ORG Ministry I-ORG 
Shen I-PER Guofang I-PER 


Sentence number: 44
Tang I-PER Shubei I-PER 
Association I-ORG for I-ORG Relations I-ORG Across I-ORG Taiwan I-ORG Straits I-ORG 


Sentence number: 48
motor I-ORG vehicles I-ORG 
Federal I-ORG office I-ORG for I-ORG 


Sentence number: 55
Opel I-ORG AG I-ORG 
General I-ORG Motors I-ORG 


Sentence number: 65
Dimitris I-PER Kontogiannis I-PER 
Athens I-ORG Newsroom I-ORG 


Sentence number: 116
Hafez I-PER al- I-PER 
David I-PER Levy I-PER 


Sentence number: 117
Benjamin I-PER Netanyahu I-PER 
Golan I-

In [92]:
# Choosing 1 sentence from the consective_named_entity_words_dict
for key, value in word_label_dict[5].items():
    print(f"{key} {value}")

Germany I-LOC
's O
representative O
to O
the O
European I-ORG
Union I-ORG
veterinary O
committee O
Werner I-PER
Zwingmann I-PER
said O
on O
Wednesday O
consumers O
should O
buy O
sheepmeat O
from O
countries O
other O
than O
Britain I-LOC
until O
scientific O
advice O
was O
clearer O
. O


As we are using the IO tagging scheme, any multi-word named entities have the same named entity word label unlike BIO which has beginning, inside and outside word label for a multi-word named entities. 
   
Hence, what we did was iterate through the sentences, and for each sentence, if the current word has a named entity word label, we will keep track of the number of subsequent consecutive words with the same named entity word label. If the number of subsequent consecutive words is more than or equal to 1, that means it is a multi-word named entity and we store the multi-word named entity in a dictionary.  
  
From the example sentence above, which is sentence number 5 in training data, there are two named entities with more than one word.  
They are:
1. European Union   
2. Werner Zwingman  

Their respective word labels are:  

European I-ORG  
Union I-ORG  
  
Werner I-PER  
Zwingmann I-PER  

